In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the CSV
df = pd.read_csv("data/company_financial_data_wrds.csv")

In [4]:
df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,ibc,invt,ni,nits,oancf,rect,sale,xint,costat,prcc_f
0,1045,12/31/2021,2021,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,-1993.0,1795.0,-1993.0,NaN,704.0,1505.0,29882.0,1800.0,A,17.96
1,1045,12/31/2022,2022,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,127.0,2279.0,127.0,NaN,2173.0,2138.0,48971.0,1962.0,A,12.72
2,1045,12/31/2023,2023,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,822.0,2400.0,822.0,NaN,3803.0,2026.0,52788.0,2145.0,A,13.74
3,1078,12/31/2021,2021,INDL,C,D,STD,ABT,ABBOTT LABORATORIES,USD,...,7071.0,5157.0,7071.0,NaN,10533.0,6487.0,43075.0,533.0,A,140.74
4,1078,12/31/2022,2022,INDL,C,D,STD,ABT,ABBOTT LABORATORIES,USD,...,6933.0,6173.0,6933.0,NaN,9581.0,6218.0,43653.0,558.0,A,109.79


In [5]:
# Convert datadate to datetime
df["datadate"] = pd.to_datetime(df["datadate"], errors="coerce")

# Filter: keep only active, consolidated, industrial format
df = df[(df["costat"] == "A") & (df["consol"] == "C") & (df["indfmt"] == "INDL")]

# Convert key numeric columns to float
numeric_cols = ['ap', 'at', 'ceq', 'che', 'cogs', 'csho', 'dltt', 'ebit', 'ebitda', 'epspx',
                'ibc', 'invt', 'ni', 'nits', 'oancf', 'rect', 'sale', 'xint', 'prcc_f']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [6]:
df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,ibc,invt,ni,nits,oancf,rect,sale,xint,costat,prcc_f
0,1045,2021-12-31,2021,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,-1993.0,1795.0,-1993.0,NaN,704.0,1505.0,29882.0,1800.0,A,17.96
1,1045,2022-12-31,2022,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,127.0,2279.0,127.0,NaN,2173.0,2138.0,48971.0,1962.0,A,12.72
2,1045,2023-12-31,2023,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,822.0,2400.0,822.0,NaN,3803.0,2026.0,52788.0,2145.0,A,13.74
3,1078,2021-12-31,2021,INDL,C,D,STD,ABT,ABBOTT LABORATORIES,USD,...,7071.0,5157.0,7071.0,NaN,10533.0,6487.0,43075.0,533.0,A,140.74
4,1078,2022-12-31,2022,INDL,C,D,STD,ABT,ABBOTT LABORATORIES,USD,...,6933.0,6173.0,6933.0,NaN,9581.0,6218.0,43653.0,558.0,A,109.79


In [7]:
# Calculate ratios — safely
df["pe_ratio"] = df["prcc_f"] / df["epspx"]
df["pb_ratio"] = df["prcc_f"] / (df["ceq"] / df["csho"])
df["ev"] = (df["csho"] * df["prcc_f"]) + df["dltt"] + df["ap"] - df["che"]
df["ev_to_ebitda"] = df["ev"] / df["ebitda"]

df["roce"] = df["ebit"] / (df["ceq"] + df["dltt"])
df["net_profit_margin"] = df["ni"] / df["sale"]

df["asset_turnover"] = df["sale"] / df["at"]
df["inventory_turnover"] = df["cogs"] / df["invt"]
df["accruals"] = df["ibc"] - df["oancf"]
df["debt_to_equity"] = df["dltt"] / df["ceq"]
df["interest_coverage"] = df["ebit"] / df["xint"]

In [8]:
df.head()

,gvkey,datadate,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,...,pb_ratio,ev,ev_to_ebitda,roce,net_profit_margin,asset_turnover,inventory_turnover,accruals,debt_to_equity,interest_coverage
0,1045,2021-12-31,2021,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,-1.584904,42165.19488,-13.263666,-0.158262,-0.066696,0.449576,15.581616,-2697.0,-5.746730,-3.063333
1,1045,2022-12-31,2022,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,-1.427171,39413.16624,9.605939,0.054451,0.002593,0.756706,16.512067,-2046.0,-6.716330,0.919980
2,1045,2023-12-31,2023,INDL,C,D,STD,AAL,AMERICAN AIRLINES GROUP INC,USD,...,-1.728126,38576.71102,6.155531,0.131488,0.015572,0.837134,16.131667,-2981.0,-6.866974,1.870862
3,1078,2021-12-31,2021,INDL,C,D,STD,ABT,ABBOTT LABORATORIES,USD,...,6.934722,260687.90068,20.848361,0.165871,0.164156,0.572836,3.224161,-3462.0,0.509804,16.821764
4,1078,2022-12-31,2022,INDL,C,D,STD,ABT,ABBOTT LABORATORIES,USD,...,5.200690,200694.51305,16.299400,0.173458,0.158821,0.586434,2.842540,-2648.0,0.421550,16.211470


In [ ]:
# Define a function to score each ratio (higher is better unless inverse=True)
def score_percentile(series, inverse=False):
    q = series.rank(pct=True)
    return (1 - q) if inverse else q

# Scoring columns
df["score_pe"] = score_percentile(df["pe_ratio"], inverse=True)
df["score_pb"] = score_percentile(df["pb_ratio"], inverse=True)
df["score_ev_ebitda"] = score_percentile(df["ev_to_ebitda"], inverse=True)

df["score_roce"] = score_percentile(df["roce"])
df["score_margin"] = score_percentile(df["net_profit_margin"])

df["score_turnover"] = score_percentile(df["asset_turnover"])
df["score_inventory"] = score_percentile(df["inventory_turnover"])
df["score_accruals"]  = score_percentile(df["accruals"], inverse=True)

df["score_dte"] = score_percentile(df["debt_to_equity"], inverse=True)
df["score_cov"] = score_percentile(df["interest_coverage"])

In [10]:
# Combine individual scores into a final composite score
score_columns = [col for col in df.columns if col.startswith("score_")]
df["fundamental_score"] = df[score_columns].mean(axis=1)

In [12]:
# Save the final dataset
df_out = df[["tic", "datadate", "fundamental_score"] + score_columns]

In [13]:
df_out

,tic,datadate,fundamental_score,score_pe,score_pb,score_ev_ebitda,score_roce,score_margin,score_turnover,score_inventory,score_accruals,score_dte,score_cov
0,AAL,2021-12-31,0.574277,0.939415,0.940736,0.981077,0.010847,0.037212,0.430311,0.560942,0.845061,0.972881,0.024283
1,AAL,2022-12-31,0.507945,0.076242,0.940054,0.803493,0.129492,0.071042,0.677267,0.568560,0.785521,0.976271,0.051508
2,AAL,2023-12-31,0.641634,0.801225,0.941417,0.911208,0.452881,0.100135,0.719892,0.563712,0.859946,0.977627,0.088300
3,ABT,2021-12-31,0.490173,0.226685,0.237057,0.289665,0.577627,0.629905,0.552774,0.219529,0.872124,0.579661,0.716703
4,ABT,2022-12-31,0.531142,0.358747,0.314714,0.471616,0.602034,0.617050,0.563599,0.189751,0.843708,0.638644,0.711553
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1787,KRG,2022-12-31,0.434539,0.995235,0.855586,NaN,0.045424,0.056834,0.108254,0.859765,0.495264,0.448136,0.046358
1789,KRG,2023-12-31,0.342721,0.038121,0.818120,NaN,0.056949,0.215832,0.113667,0.859765,0.471583,0.450847,0.059603
1790,NVT,2021-12-31,0.460833,0.467665,0.579700,0.483261,0.329492,0.438430,0.505413,0.268006,0.290934,0.633220,0.612215
1791,NVT,2022-12-31,0.517049,0.648741,0.616485,0.610626,0.386441,0.537889,0.569012,0.308172,0.169824,0.643390,0.679912


In [ ]:
df_out.to_csv("data/fundamental_scores_wrds.csv", index=False)